# Use Case: 4LAC

In this notebook we'll use BlaSE to reestimate the 4LAC-DR2 catalogue. The values for the synchrotron peak are stored in `table-4LAC-DR2-h.fits` and `table-4LAC-DR2-l-fits`, which both originates from https://fermi.gsfc.nasa.gov/ssc/data/access/lat/4LACDR2/. See the link for more information. Additionally, the VOUBlazar tool was used to get the actual SEDs which are stored in `4LAC.zip`.

In [ ]:
#You need to have blase installed
!blase 4LAC.zip -o 4LAC_estimates.csv

In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits

output = pd.read_csv('4LAC_estimates.csv')
fits_h = fits.open('table-4LAC-DR2-h.fits')
dh = fits_h[1].data
fits_l = fits.open('table-4LAC-DR2-l.fits')
dl = fits_l[1].data

cat_ra = np.concatenate((dh.field('RA_Counterpart'), dl.field('RA_Counterpart')))
cat_dec = np.concatenate((dh.field('DEC_Counterpart'), dl.field('DEC_Counterpart')))
cat_syn = np.concatenate((dh.field('nu_syn  '), dl.field('nu_syn  ')))

fits_h.close()
fits_l.close()

print(f'{output.shape[0]} seds loaded.')
print(f'{len(cat_ra)} catalogue entries loaded.')
print(f'{(cat_syn == 0.0).sum()} missing synchrotron peaks.')

3696 seds loaded.
3511 catalogue entries loaded.
917 missing synchrotron peaks.


In [2]:
from scipy.spatial import KDTree

#convert to log10
mask = cat_syn != 0.0
cat_syn[mask] = np.log10(cat_syn[mask])

#create kdtree to match catalogue with blase's estimations
cat_pos = np.stack((cat_ra, cat_dec), axis=1)
tree = KDTree(cat_pos)

In [3]:
#kdtree will return an index one after the last for missing values
#we take adventage by populating that index with the "missing value"-value
syn = np.append(cat_syn, [0.0])

#match with catalogue
_, indices = tree.query(output[['Right Ascension', 'Declination']], distance_upper_bound=0.1, workers=-1)
output['Catalogue Peak'] = syn[indices]

In [4]:
from blase import get_bag

#Some seds of the 4LAC were part of the training set. Check which.
bags = get_bag(output[['Right Ascension', 'Declination']])
output['Unseen'] = (bags == -1)

print(f'{(bags == -1).sum()} unseen data.')
print(f'{((bags == -1) & (output["Catalogue Peak"] == 0.0)).sum()} unseen and missing peaks.')

1053 unseen data.
706 unseen and missing peaks.


In [5]:
#map catalogue peak to string to force two decimal points
output['Catalogue Peak'] = output['Catalogue Peak'].map(lambda x: f'{x:.2f}')
#reorder columns
output = output.iloc[:,[0,1,2,5,6,3,4]] #catalogue before estimation
#Save results
output.to_csv('4LAC.csv', index=False)